# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function based on some assumptions about our business. 

In [ ]:
import evalml
from evalml.objectives import FraudDetection
from evalml.preprocessing import split_data

## Load Transactions

In [ ]:
X, y = evalml.demos.load_fraud()

## Define Objective

In [ ]:
fraud_objective = FraudDetection(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [ ]:
# select numeric data before running AutoClassifer
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [ ]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=10)

clf.fit(X_train, y_train)

### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [ ]:
clf.rankings

to select the best pipeline we can run

In [ ]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [ ]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [ ]:
clf.describe_pipeline(0)

## Test on hold out

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
pipeline.score(X_holdout, y_holdout)